In [1]:
import os
import transformers
import torch
import torch.nn.functional as F
import json
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score

In [ ]:
# Set the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model and tokenizer
detector = transformers.AutoModelForSequenceClassification.from_pretrained("TrustSafeAI/RADAR-Vicuna-7B")
tokenizer = transformers.AutoTokenizer.from_pretrained("TrustSafeAI/RADAR-Vicuna-7B")
detector.eval()
detector.to(device)

In [3]:
# Function to classify text and write results to output file
def classify_text(text):
    with torch.no_grad():
        inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        output_probs = F.log_softmax(detector(**inputs).logits, -1)[:, 0].exp().item()

    # Classify as human-written or AI-generated based on probability threshold
    if output_probs < 0.5:
        total_pred = 0
    else:
        total_pred = 1

    return total_pred

In [13]:
def predictor(ai_iclr, ai_neur, human_iclr, human_neur):
    """For prediction from the model"""
    ai_iclr = list(json.load(open(ai_iclr)).values())
    ai_neur = list(json.load(open(ai_neur)).values())
    human_iclr = list(json.load(open(human_iclr)).values())
    human_neur = list(json.load(open(human_neur)).values())

    ai_iclr_pred = [classify_text(text) for text in ai_iclr[608:]]
    ai_neur_pred = [classify_text(text) for text in ai_neur[576:]]
    human_iclr_pred = [classify_text(a) for text in human_iclr[608:] for a in text]
    human_neur_pred = [classify_text(a) for text in human_neur[576:] for a in text]

    actual = [1 for i in ai_iclr[608:]]  + [0 for revw in human_iclr[608:] for i in revw]
    predicted = ai_iclr_pred + human_iclr_pred

    print("++++++++++++++++ ICLR ++++++++++++++++")
    print("Confusion Matrix: ", confusion_matrix(actual, predicted))
    print("F1 Score: ", f1_score(actual, predicted))
    print("Precision: ", precision_score(actual, predicted))
    print("Recall: ", recall_score(actual, predicted))

    actual = [1 for i in ai_neur[576:]]  + [0 for revw in human_neur[576:] for i in revw]
    predicted = ai_neur_pred + human_neur_pred
    print("\n")
    print("++++++++++++++++ NEUR ++++++++++++++++")
    print("Confusion Matrix: ", confusion_matrix(actual, predicted))
    print("F1 Score: ", f1_score(actual, predicted))
    print("Precision: ", precision_score(actual, predicted))
    print("Recall: ", recall_score(actual, predicted))

## Actual Reviews:-

In [ ]:
predictor("Dataset/ai_iclr.json","Dataset/ai_neur.json","Dataset/human_iclr.json","Dataset/human_neur.json")

## Synonym Attack (Adjective Attack):-

In [ ]:
predictor("Dataset/Attacked Dataset/Adjective Attack/ai_iclr.json","Dataset/Attacked Dataset/Adjective Attack/ai_neur.json","Dataset/Attacked Dataset/Adjective Attack/human_iclr.json","Dataset/Attacked Dataset/Adjective Attack/human_neur.json")

## Paraphrasing Attack:-

In [ ]:
predictor("Dataset/Attacked Dataset/Paraphrasing Attack/para_ai_iclr.json","Dataset/Attacked Dataset/Paraphrasing Attack/para_ai_neur.json","Dataset/Attacked Dataset/Paraphrasing Attack/para_human_iclr.json","Dataset/Attacked Dataset/Paraphrasing Attack/para_human_neur.json")

## Modified Paraphrasing Attack:-

In [ ]:
predictor("Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Paraphrased/ai_iclr_para_modified.json","Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Paraphrased/ai_neur_para_modified.json","Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Paraphrased/human_iclr_para_modified.json","Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Paraphrased/human_neur_para_modified.json")